In [1]:
#Loading gensim and nltk libraries
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
np.random.seed(2018)
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Min\Anaconda3\lib\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
stopWords = ['author', 'virus', 'wuhan', 'coronavirus', '2019nCoV', 'Covid-19', 'covid', 'singapore', 'wuhan pneumonia', 'novel coronavirus', 'corona virus', 'wuhanvirus', 'https', 'http', 'leea']

In [3]:
#method for lemmatization and stemming
def preprocess(text):
    try:
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                if token not in stopWords:
                    #result.append(lemmatize_stemming(token))
                    result.append(token)
        #print(result)
        return result
    except:
        return []
def lemmatize_stemming(text):
    stemmer = SnowballStemmer("english")
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


In [4]:
posts = pd.read_csv('covid_Reddit_output_21_03_2020.csv')
posts = posts.replace(np.nan, ' ', regex=True)
posts.head(3)

,author,title,score,id,url,comms_num,created,body,timestamp
0,satoshigekkouga2303,Singapore reports two deaths from Covid-19,1201,fm8vm4,https://www.straitstimes.com/singapore/singapo...,254,1.584791e+09,,2020-03-21 19:50:47
1,SadKaleidoscope2,"[Update] SG COVID-19 cases by cluster, as of 2...",83,fm6r6m,https://i.redd.it/qn3182yvfxn41.png,6,1.584782e+09,,2020-03-21 17:16:26
2,justmewayne,PASSING OF TWO PATIENTS WITH COVID-19 INFECTION,63,fm90ns,https://www.moh.gov.sg/news-highlights/details...,30,1.584792e+09,,2020-03-21 20:01:51


In [6]:
posts['combined_body'] = posts['title'] + ' ' + posts['body'] + ' ' 
# + posts['top_main_comment']

In [7]:
posts['combined_body'][2]

'PASSING OF TWO PATIENTS WITH COVID-19 INFECTION   '

In [8]:
posts['combined_body'] = posts['combined_body'].map(preprocess)
print('Pre-processing is done.')

Pre-processing is done.


In [9]:
#building dictionary and corpus for LDA
dictionary = gensim.corpora.Dictionary(posts['combined_body'])
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in posts['combined_body']]
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 20:
        break
print('Dictionary is built.')

Dictionary(2431 unique tokens: ['covid', 'deaths', 'reports', 'singapore', 'cases']...)
0 covid
1 deaths
2 reports
3 singapore
4 cases
5 cluster
6 update
7 infection
8 passing
9 patients
10 airline
11 airlines
12 announced
13 anymore
14 barrier
15 block
16 blocking
17 bringing
18 cause
19 considering
20 curious
Dictionary is built.


In [10]:
#Generate Bag of Words on the data set
bow_corpus = [dictionary.doc2bow(text) for text in posts['combined_body']]
#generate TFIDF matrix
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.00805468636067354),
 (1, 0.7717216829654875),
 (2, 0.6181712120032168),
 (3, 0.1491479759132541)]


In [11]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step, coherence):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence=coherence)
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [12]:
start = 5
limit = 41
step = 5

try:
    model_list, coherence_values_umass = compute_coherence_values(dictionary=dictionary, 
                                                        corpus=corpus, texts=posts['combined_body'], 
                                                        start=start, limit=limit, step=step, coherence='u_mass')
except Exception as e:
    print(e)

In [13]:
start = 5
limit = 41
step = 5
# Show graph
import matplotlib.pyplot as plt
x = range(start, limit, step)
plt.plot(x, coherence_values_umass)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score for u_mass")
plt.legend(("coherence_values"), loc='best')
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
#run LDA
#change num_topics if want to change the number of topics generated
lda_model = gensim.models.LdaModel(corpus, num_topics=8, id2word=dictionary, random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)


In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.041*"know" + 0.017*"covid" + 0.015*"airlines" + 0.014*"tickets" + 0.013*"thanks" + 0.013*"counts" + 0.013*"queued" + 0.012*"flights" + 0.010*"real" + 0.010*"recently"
Topic: 1 
Words: 0.103*"covid" + 0.064*"singapore" + 0.039*"discharged" + 0.029*"infection" + 0.022*"confirmed" + 0.016*"student" + 0.012*"confirms" + 0.010*"couple" + 0.010*"ward" + 0.009*"restaurant"
Topic: 2 
Words: 0.051*"singapore" + 0.019*"covid" + 0.015*"malaysia" + 0.014*"going" + 0.013*"supply" + 0.011*"travel" + 0.011*"near" + 0.011*"getting" + 0.010*"https" + 0.010*"south"
Topic: 3 
Words: 0.021*"countries" + 0.021*"case" + 0.017*"think" + 0.015*"food" + 0.014*"sure" + 0.013*"month" + 0.013*"work" + 0.012*"chinese" + 0.011*"world" + 0.011*"currently"
Topic: 4 
Words: 0.079*"people" + 0.064*"covid" + 0.063*"cases" + 0.031*"feel" + 0.017*"safra" + 0.015*"cluster" + 0.014*"says" + 0.013*"jurong" + 0.013*"including" + 0.011*"linked"
Topic: 5 
Words: 0.051*"situation" + 0.044*"covid" + 0.026*"test

In [16]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

C:\Users\Min\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.133275 -0.182537       1        1  17.255737
3      0.196297  0.160556       2        1  16.561850
7     -0.087749 -0.020608       3        1  14.644673
2     -0.051840  0.021160       4        1  14.614290
5     -0.057940  0.002471       5        1  10.875848
4     -0.062526  0.042082       6        1   9.274734
1     -0.095809  0.037003       7        1   8.586274
0      0.026291 -0.060126       8        1   8.186591, topic_info=     Category        Freq          Term       Total  loglift  logprob
0     Default  200.000000         covid  200.000000  30.0000  30.0000
3     Default   93.000000     singapore   93.000000  29.0000  29.0000
61    Default   46.000000        people   46.000000  28.0000  28.0000
4     Default   45.000000         cases   45.000000  27.0000  27.0000
128   Default   50.000000          like   50.000000  26.0000  26.0000
...       ...         ...           ...         ...      ...      ...
85     Topic8    2.896597        impact    3.698449   2.2583  -5.1646
1525   Topic8    2.811839  cancellation    3.615293   2.2513  -5.1943
546    Topic8    2.669194          risk    3.468801   2.2406  -5.2464
416    Topic8    3.225013        family    5.563258   1.9574  -5.0572
0      Topic8    8.813529         covid  200.772049  -0.6232  -4.0519

[293 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
494       7  0.714750  accepting
102       1  0.853714   actually
388       5  0.834838    advised
2318      8  0.909848  aesthetic
103       1  0.856782   affected
...     ...       ...        ...
53        2  0.944756       work
2423      6  0.835252  workforce
1662      8  0.866051    working
273       2  0.946140      world
169       1  0.916414       year

[267 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 8, 3, 6, 5, 2, 1])